In [ ]:
from laserembeddings import Laser
import pandas as pd
from nltk.tokenize import sent_tokenize
import nltk
import fasttext
import numpy as np
import seaborn as sns

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

from sklearn.feature_extraction.text import TfidfVectorizer
import fasttext

import pickle
import csv

In [ ]:
!python -m laserembeddings download-models

In [ ]:
fb = pd.read_csv("facebook_cc.csv", engine='python')

In [ ]:
fasttext_model = fasttext.load_model('lid.176.bin')

In [ ]:
fb = fb.dropna(subset = ['Message'] ).reset_index(drop = True)

docs = fb['Message'].to_list()

text = fb['Message']


In [ ]:

#remove html markup
text = text.str.replace('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});',' ')
# 
#remove tabs
text= text.str.replace('[ |\t]{2,}', ' ', case=False, regex = True)

#remove urls 
text= text.str.replace('http\S+|www\.\S+', ' ', case=False, regex = True)

#remove urls 
text= text.str.replace('\S+\.se|\S+\.com|\S+\.org', ' ', case=False, regex = True)

#remove e-mails
text= text.str.replace('\S+@\S+', ' ', case=False, regex = True)

#remove additional links
text= text.str.replace('bit\.ly\S+', ' ', case=False, regex = True)
text= text.str.replace('goo\.gl\S+', ' ', case=False, regex = True)
text= text.str.replace('youtu\.be\S+', ' ', case=False, regex = True)

#remove numbers and punctuation EXCEPT FOR ! ? , .
text = text.str.replace('[^a-zåöäéáúğüíæøýóç!?,.-: ]',' ', regex = True).str.replace(' +',' ').str.strip()



In [ ]:
docs_lang_fasttext = list(map(fasttext_model.predict, docs))

In [ ]:
docs_lang_fasttext_labels = [x[0] for x in docs_lang_fasttext]

docs_lang_fasttext_labels = [''.join(x) for x in docs_lang_fasttext_labels] 
docs_lang_fasttext_labels = [x[-2:] for x in docs_lang_fasttext_labels]


#merge into one df and remove non-sv documents

fb['fasttext'] = docs_lang_fasttext_labels

fb = fb[(fb['fasttext'] == "sv") | (fb['fasttext'] == "no") | (fb['fasttext'] == "da" ) | (fb['fasttext'] == "fi" ) | (fb['fasttext'] == "en" )]
fb = fb.reset_index(drop = True)


In [ ]:
fb = fb.dropna(subset=['Message'])
fb=fb[fb.Message != '']

docs = fb['Message'].to_list()
len(docs)

In [ ]:
column_ids = []
for i in range (0, 1024):
    column_ids.append(str(i))


In [ ]:
laser = Laser()

embeddings = []
    
for i in range(100001, len(docs)):


    b = laser.embed_sentences(docs[i], lang = fb['fasttext'][i]) 
    embeddings.append(b[0])
    
    if (i >0 )& (i % 100000 == 0):
         
        df = pd.DataFrame(embeddings, columns = column_ids) 
        df.to_csv("C:/Users/vicya449/Box/embeddings" + str(i) + ".csv")
        print(len(df))
        embeddings = []
        
        
    elif (i == (len(docs)-1)):
        
        df = pd.DataFrame(embeddings, columns = column_ids) 
        df.to_csv("C:/Users/vicya449/Box/embeddings" + str(i) + ".csv")